In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import sys
sys.path.append("../../")
import seq2seq

In [2]:
# chess

with open("data/chess_book.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [3]:
len(text)

556949

In [4]:
vocabulary = set(text)
len(vocabulary)

92

In [5]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
print(len(char2i))
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"
print(len(i2char))

95
95


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [7]:
length = 20
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])
encoded = torch.tensor([[char2i[c] for c in l] for l in tqdm(lines)]).to(device).long()
print(encoded.shape)
source_1 = encoded[:, :length // 2]
print(source_1.shape)
target_1 = torch.cat((torch.ones(encoded.shape[0], 1).to(device).long(), encoded[:, length // 2:]), axis = 1)
print(target_1.shape)

556949
556929
['\ufeffThe Project Gutenbe', 'The Project Gutenber', 'he Project Gutenberg', 'e Project Gutenberg ', ' Project Gutenberg E']



torch.Size([556929, 20])
torch.Size([556929, 10])
torch.Size([556929, 11])


In [13]:
importlib.reload(seq2seq)  

net = seq2seq.Transformer(char2i, i2char)
net.to(device)
performance = net.fit(source_1, target_1, save_path = "checkpoints/seq2seq_transformer.pt")
net.save_architecture("architectures/seq2seq_transformer.architecture")

Model: Transformer
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
Max sequence length: 32
Embedding dimension: 64
Feedforward dimension: 256
Encoder layers: 2
Decoder layers: 2
Attention heads: 2
Activation: relu
Dropout: 0.0
Trainable parameters: 254,111

Epoch | Train                 | Training time
      | Loss     | Error Rate |
---------------------------------------------
    1 |   1.5897 |     42.718 |        112.53
    2 |   1.2019 |     35.141 |        224.61
    3 |   1.1139 |     33.039 |        336.76
    4 |   1.0630 |     31.822 |        447.15
    5 |   1.0275 |     30.963 |        557.93


In [14]:
importlib.reload(seq2seq)    
net = seq2seq.load_architecture("architectures/seq2seq_transformer.architecture")
net.load_state_dict(torch.load("checkpoints/seq2seq_transformer.pt"))
net.to(device)
indexes, log_probabilities = net.greedy_search(source_1[:5], max_predictions = 30)

print(log_probabilities)
net.tensor2text(indexes)

Model: Transformer
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
Max sequence length: 32
Embedding dimension: 64
Feedforward dimension: 256
Encoder layers: 2
Decoder layers: 2
Attention heads: 2
Activation: relu
Dropout: 0.0
Trainable parameters: 254,111

tensor([-22.2669, -24.0849, -27.8247, -28.5373, -25.4685], device='cuda:0')


['<START>ct Gutenbedg picedg. GutGuc mu',
 '<START>t Gutenberges sedgRonititic mu',
 '<START> GutenbergGa P digR4.orty t mu',
 '<START>of the pawis.poFGEswetrtlaw mp',
 '<START>utenberg-t4 GmiEGEmamututim m ']

In [15]:
importlib.reload(seq2seq)    
net = seq2seq.load_architecture("architectures/seq2seq_transformer.architecture")
net.load_state_dict(torch.load("checkpoints/seq2seq_transformer.pt"))
net.to(device)
indexes, log_probabilities = net.sample(source_1[:5])

print(log_probabilities)
net.tensor2text(indexes)

Model: Transformer
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
Max sequence length: 32
Embedding dimension: 64
Feedforward dimension: 256
Encoder layers: 2
Decoder layers: 2
Attention heads: 2
Activation: relu
Dropout: 0.0
Trainable parameters: 254,111

tensor([-51.6691, -27.4829, -30.3443, -35.4172, -49.7243], device='cuda:0')


['<START> GLAve thejd. KRkAM4',
 '<START>t Guter usua piftndd',
 '<START> of White, p p ams w',
 '<START>Black King\nwIm powag',
 '<START>eng 20\nwat50 K  KtRn']

In [16]:
importlib.reload(seq2seq)    
net = seq2seq.load_architecture("architectures/seq2seq_transformer.architecture")
net.load_state_dict(torch.load("checkpoints/seq2seq_transformer.pt"))
net.to(device)
indexes, log_probabilities = net.beam_search(source_1[:5])

print(log_probabilities)
[net.tensor2text(t) for t in indexes]

Model: Transformer
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
Max sequence length: 32
Embedding dimension: 64
Feedforward dimension: 256
Encoder layers: 2
Decoder layers: 2
Attention heads: 2
Activation: relu
Dropout: 0.0
Trainable parameters: 254,111

tensor([[-12.6637, -12.7309, -12.7573, -13.4648, -13.5960],
        [-13.4404, -13.7053, -13.8982, -13.9962, -14.0315],
        [-13.6261, -13.6618, -14.9730, -15.2090, -15.4287],
        [-17.3752, -17.6654, -18.0157, -18.4944, -18.7397],
        [-16.0127, -16.1489, -17.1593, -17.7551, -17.9944]], device='cuda:0')


[['<START>ct Gutenbedg picedg.',
  '<START>ct Gutenbedg picedga',
  '<START>ct Gutenbedg picedd ',
  '<START>ct Gutenbedg picedgo',
  '<START>ct Gutenbedg, sEGgR3'],
 ['<START>t Gutenberges sedid ',
  '<START>t Gutenbergesssemid ',
  '<START>t Gutenbergesssemidt',
  '<START>t Gutenberges sedgRo',
  '<START>t Gutenberges sedidt'],
 ['<START> Gutenberggest ceddo',
  '<START> Gutenberedest ceddo',
  '<START> Gutenberggest cedd4',
  '<START> Gutenberggest cedd ',
  '<START> Gutenbergges sedgRo'],
 ['<START>of the BlaG| Kt Q4Kt',
  '<START>of the BlaG| Kt KKKt',
  '<START>of the BlaG| Kt KKtt',
  '<START>of the pawis.poFGEsw',
  '<START>of the pawis.poFGEst'],
 ['<START>utenberg-t4 GmiM, ma',
  '<START>utenberg-t4 GmiM, Kt',
  '<START>utenberg-t4 GmiM, G ',
  '<START>utenberg-t4 GmiM, R4',
  '<START>utenberg-t4 GmiM, d.']]

In [17]:
importlib.reload(seq2seq)    
net = seq2seq.load_architecture("architectures/seq2seq_transformer.architecture")
net.load_state_dict(torch.load("checkpoints/seq2seq_transformer.pt"))
net.to(device)

idx, log_probabilities = net.predict(source_1[:5])

net.tensor2text(idx)

Model: Transformer
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
Max sequence length: 32
Embedding dimension: 64
Feedforward dimension: 256
Encoder layers: 2
Decoder layers: 2
Attention heads: 2
Activation: relu
Dropout: 0.0
Trainable parameters: 254,111



['<START>ct Gutenbe',
 '<START>t Gutenber',
 '<START> Gutenberg',
 '<START>of the Bla',
 '<START>utenberg-t']